In [21]:
import math, json, random
import sat_ai_core
from explain_ai import explain_answer
from IPython.display import display, Markdown

In [26]:
display(Markdown("## 🧠 Bắt đầu mô phỏng thi thích ứng SAT-AI"))
theta = 0.0
asked = []
history = []

for step in range(5):
    item = sat_ai_core.select_next_item(theta, asked)
    if not item:
        display(Markdown("✅ Hết câu hỏi trong ngân hàng!"))
        break

    asked.append(item["id"])
    display(Markdown(f"### Câu {step+1}: {item['question']}"))
    for i, c in enumerate(item["choices"]):
        print(f"{i+1}. {c}")

    ans = input("Đáp án của bạn (1–4 hoặc 'q' để thoát): ").strip()
    if ans.lower() == "q":
        print("🛑 Kết thúc sớm.")
        break

    correct = int(ans) - 1 == item["answer_index"]
    print("✅ Đúng!" if correct else "❌ Sai.")
    # Robust lookup for IRT params: try str key, then int key, then raw key, then fallback default
    key_str = str(item["id"])
    params = None
    # sat_ai_core may expose irt_params with string keys (common) or numeric keys; try both
    try:
        params = sat_ai_core.irt_params.get(key_str)
    except Exception:
        params = None
    if params is None:
        try:
            params = sat_ai_core.irt_params.get(int(item["id"]))
        except Exception:
            params = None
    if params is None:
        params = sat_ai_core.irt_params.get(item["id"])
    if params is None:
        # If still missing, show a helpful message and use safe defaults
        display(Markdown(f"⚠️ Không tìm thấy tham số IRT cho câu hỏi id={item['id']} — sử dụng mặc định a=1.0, b=0.0, c=0.25"))
        params = {"a": 1.0, "b": 0.0, "c": 0.25}
    theta = sat_ai_core.update_theta(theta, int(correct),
                                     params["a"], params["b"], params["c"])
    print(f"Năng lực hiện tại θ = {theta:.2f}\n")
    history.append((item["id"], correct, theta))

display(Markdown(f"### 🎯 Năng lực cuối cùng (θ_final): **{theta:.2f}**"))


## 🧠 Bắt đầu mô phỏng thi thích ứng SAT-AI

KeyError: 3